<a href="https://colab.research.google.com/github/carloscariello/AED_TrabPratico1/blob/master/Pagamento_Primeira_Parcela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar a biblioteca lifelines
!pip install lifelines



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=fcc4f7a97b24e44111674a30da6a31753b8fea88107b4653e051e764903f2fb2
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [2]:
# Importar bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter



In [26]:
# Supondo que você tenha seu arquivo salvo no Colab, carregue o arquivo de dados
# Caso o arquivo esteja no seu Google Drive, você pode usar:
# from google.colab import drive
# drive.mount('/content/drive')
# file_path = '/content/drive/MyDrive/output(1).xlsx'
file_path = '/content/dados_brutos.xlsx'

# Carregar os dados
df = pd.read_excel(file_path, sheet_name='Sheet1')

df.head()

,ts_acrd,qt_pcl_acrd,dt_lqdc_sdo_pcl,pagou_parcela,cd_nh_mtu_dgtl,cd_nh_fun_pbco,vl_sdo_vcd,vl_edv_ttl,vl_edv_ttl_sfn,cd_sexo,nr_dd_atr_cli
0,2023-12-28 11:25:59.304208,1,2024-01-05 00:00:00,1,NaN,NaN,135779.71,135779.71,126631.56,M,582
1,2024-01-11 09:38:29.560489,1,2024-01-15 00:00:00,1,NaN,NaN,15558.00,524878.52,NaN,NaN,155
2,2024-01-05 11:09:18.886185,1,2024-02-06 00:00:00,1,NaN,NaN,3647.25,3647.25,2953.95,F,548
3,2023-12-27 12:36:46.265623,11,2024-02-09 00:00:00,1,7.0,0.0,6477.87,6573.04,263.89,M,268
4,2024-01-02 10:58:02.646534,1,2024-01-02 00:00:00,1,NaN,NaN,24522.22,103609.45,108486.15,M,1438


In [27]:
# Limpeza de dados e preenchimento de valores faltantes
df['cd_nh_mtu_dgtl'].fillna(1, inplace=True)
df['cd_nh_fun_pbco'].fillna(0, inplace=True)

# Criar a razão do saldo vencido sobre o endividamento
df['ratio_sdo_vcd_edv'] = df['vl_sdo_vcd'] / df['vl_edv_ttl']

# Transformar o gênero em variáveis dummy
df = pd.get_dummies(df, columns=['cd_sexo'], drop_first=True)

# Remover linhas com valores nulos em 'nr_dd_atr_cli'
df.dropna(subset=['nr_dd_atr_cli'], inplace=True)

# Inverter o evento de interesse (não pagou parcela será o evento)
df['nao_pagou_parcela'] = 1 - df['pagou_parcela']

df.head()


,ts_acrd,qt_pcl_acrd,dt_lqdc_sdo_pcl,pagou_parcela,cd_nh_mtu_dgtl,cd_nh_fun_pbco,vl_sdo_vcd,vl_edv_ttl,vl_edv_ttl_sfn,nr_dd_atr_cli,ratio_sdo_vcd_edv,cd_sexo_M,nao_pagou_parcela
0,2023-12-28 11:25:59.304208,1,2024-01-05 00:00:00,1,1.0,0.0,135779.71,135779.71,126631.56,582,1.000000,True,0
1,2024-01-11 09:38:29.560489,1,2024-01-15 00:00:00,1,1.0,0.0,15558.00,524878.52,NaN,155,0.029641,False,0
2,2024-01-05 11:09:18.886185,1,2024-02-06 00:00:00,1,1.0,0.0,3647.25,3647.25,2953.95,548,1.000000,False,0
3,2023-12-27 12:36:46.265623,11,2024-02-09 00:00:00,1,7.0,0.0,6477.87,6573.04,263.89,268,0.985521,True,0
4,2024-01-02 10:58:02.646534,1,2024-01-02 00:00:00,1,1.0,0.0,24522.22,103609.45,108486.15,1438,0.236679,True,0


In [28]:
from sklearn.preprocessing import StandardScaler

# Definir as colunas numéricas que precisam ser escalonadas
numeric_cols = ['nr_dd_atr_cli', 'qt_pcl_acrd', 'ratio_sdo_vcd_edv', 'cd_nh_mtu_dgtl']

# Inicializar o escalador
scaler = StandardScaler()

# Aplicar o escalonamento apenas nas colunas numéricas
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df.head()


,ts_acrd,qt_pcl_acrd,dt_lqdc_sdo_pcl,pagou_parcela,cd_nh_mtu_dgtl,cd_nh_fun_pbco,vl_sdo_vcd,vl_edv_ttl,vl_edv_ttl_sfn,nr_dd_atr_cli,ratio_sdo_vcd_edv,cd_sexo_M,nao_pagou_parcela
0,2023-12-28 11:25:59.304208,-0.606937,2024-01-05 00:00:00,1,-0.376325,0.0,135779.71,135779.71,126631.56,0.120188,0.818617,True,0
1,2024-01-11 09:38:29.560489,-0.606937,2024-01-15 00:00:00,1,-0.376325,0.0,15558.00,524878.52,NaN,-0.452119,-1.812664,False,0
2,2024-01-05 11:09:18.886185,-0.606937,2024-02-06 00:00:00,1,-0.376325,0.0,3647.25,3647.25,2953.95,0.074618,0.818617,False,0
3,2023-12-27 12:36:46.265623,-0.226460,2024-02-09 00:00:00,1,2.420938,0.0,6477.87,6573.04,263.89,-0.300666,0.779355,True,0
4,2024-01-02 10:58:02.646534,-0.606937,2024-01-02 00:00:00,1,-0.376325,0.0,24522.22,103609.45,108486.15,1.267483,-1.251247,True,0


In [29]:
import numpy as np


# Remover outliers com base em um limite específico (exemplo: mais de 3 desvios padrão)
df = df[(np.abs(df['ratio_sdo_vcd_edv'] - df['ratio_sdo_vcd_edv'].mean()) <= (3 * df['ratio_sdo_vcd_edv'].std()))]

df.head()


,ts_acrd,qt_pcl_acrd,dt_lqdc_sdo_pcl,pagou_parcela,cd_nh_mtu_dgtl,cd_nh_fun_pbco,vl_sdo_vcd,vl_edv_ttl,vl_edv_ttl_sfn,nr_dd_atr_cli,ratio_sdo_vcd_edv,cd_sexo_M,nao_pagou_parcela
0,2023-12-28 11:25:59.304208,-0.606937,2024-01-05 00:00:00,1,-0.376325,0.0,135779.71,135779.71,126631.56,0.120188,0.818617,True,0
1,2024-01-11 09:38:29.560489,-0.606937,2024-01-15 00:00:00,1,-0.376325,0.0,15558.00,524878.52,NaN,-0.452119,-1.812664,False,0
2,2024-01-05 11:09:18.886185,-0.606937,2024-02-06 00:00:00,1,-0.376325,0.0,3647.25,3647.25,2953.95,0.074618,0.818617,False,0
3,2023-12-27 12:36:46.265623,-0.226460,2024-02-09 00:00:00,1,2.420938,0.0,6477.87,6573.04,263.89,-0.300666,0.779355,True,0
4,2024-01-02 10:58:02.646534,-0.606937,2024-01-02 00:00:00,1,-0.376325,0.0,24522.22,103609.45,108486.15,1.267483,-1.251247,True,0


In [30]:
# Selecionar as colunas para o modelo
cox_data = df[['nao_pagou_parcela', 'nr_dd_atr_cli', 'cd_nh_mtu_dgtl', 'ratio_sdo_vcd_edv', 'cd_sexo_M']]

# Dividir os dados em treinamento (80%) e teste (20%)
train_data, test_data = train_test_split(cox_data, test_size=0.2, random_state=42)



In [31]:
# Inicializar e ajustar o modelo de Cox
cph = CoxPHFitter()
cph.fit(train_data, duration_col='nr_dd_atr_cli', event_col='nao_pagou_parcela')



<lifelines.CoxPHFitter: fitted with 4000 total observations, 3684 right-censored observations>

In [32]:

# Mostrar o sumário do modelo de Cox
cph.print_summary()


<lifelines.CoxPHFitter: fitted with 4000 total observations, 3684 right-censored observations>
             duration col = 'nr_dd_atr_cli'
                event col = 'nao_pagou_parcela'
      baseline estimation = breslow
   number of observations = 4000
number of events observed = 316
   partial log-likelihood = -2278.43
         time fit was run = 2024-09-21 00:52:08 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
cd_nh_mtu_dgtl      0.35       1.42       0.05             0.25             0.45                 1.29                 1.56
ratio_sdo_vcd_edv  -0.79       0.45       0.06            -0.91            -0.67                 0.40                 0.51
cd_sexo_M          -0.03       0.97       0.11            -0.25             0.19                 0.78                 1.21

                    cmp to      z      p   -log2(p)
covariate                                          
cd_nh_mtu_dgtl        0.00   7.05 <0.005      38.99
ratio_sdo_vcd_edv     0.00 -12.94 <0.005     124.76
cd_sexo_M             0.00  -0.28   0.78       0.36
---
Concordance = 0.80
Partial AIC = 4562.86
log-likelihood ratio test = 273.45 on 3 df
-log2(p) of ll-ratio test = 193.53

In [33]:

# Avaliar o modelo usando os dados de teste
cph.predict_median(test_data)



,0.5
1501,0.354740
2586,0.486089
2653,inf
1055,inf
705,inf
...,...
4711,inf
2313,0.772913
3214,inf
2732,inf


In [34]:



# Para visualizar as previsões de risco para os dados de teste:
risks = cph.predict_partial_hazard(test_data)
print(risks)

1501    4.593894
2586    3.697058
2653    0.396190
1055    0.452046
705     0.452046
          ...   
4711    0.452046
2313    3.291707
3214    0.466437
2732    0.452046
1926    0.577569
Length: 1000, dtype: float64


In [23]:
# Exibir os coeficientes de risco proporcional para cada variável
print(cph.params_)

covariate
cd_nh_mtu_dgtl       0.163229
ratio_sdo_vcd_edv   -0.791099
cd_sexo_M           -0.031339
Name: coef, dtype: float64
